#  RESEARCH PROJECT B
Student: Mr Eloy Ruiz Donayre  
Supervisor: Prof Dr Achim Kehrein  

Reference Paper:  
> To sleep or not to sleep: the ecology of sleep in artificial organisms  
> Alberto Acerbi, Patrick McNamara and Charles L Nunn  
> https://doi.org/10.1186/1472-6785-8-10 

## Testing a full experiment (15 generations)

### Setting up modules

In [1]:
%matplotlib inline
# from jupyterthemes import jtplot
# jtplot.style()

In [2]:
import math
import random
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from simulation_modules.agents import Animal, FoodPatch, SleepPatch
from simulation_modules.model import SleepAnimals
from simulation_modules.experiment import *

In [3]:
current_exp_id = 0
generation_number = 0
abm_models = []
generations = []
results = pd.DataFrame()
p = 0.05

### Creating genomes of initial population
The array `genomes` represent the genomes of the first 100 individuals. Each gene in the genome has three possible values:  
`'flex' , 'eat' , 'sleep'`

In [4]:
genomes = genome_alternatives(100)
np.shape(genomes)
selected_genomes = genomes

##### Executing 20 generations

In [5]:
%%time
for run_generations in range (5):
    # Creating the Agent-Based models for the current generation and running it for 7 days.
    for current_exp_id in range(100*generation_number + 0, 100*generation_number + 100):
        i = current_exp_id
        abm_models.append( SleepAnimals(i, genome=selected_genomes[i-100*generation_number], 
                                        width=40, height=40) )
        for j in range(60*24*7):
            abm_models[i].step()
    
    # Capturing the data from the end of the simulation
    current_generation = []
    for k in range(100*generation_number + 0, 100*generation_number + 100):
        a = [ abm_models[k].model_id , abm_models[k].schedule.agents[0].fitness , 
             abm_models[k].schedule.agents[0].circadian_rythm ]
        current_generation.append( a )
    
    generation_results = dataframe_generation(generation_number , current_generation)
    results = results.append(generation_results)
    current_generation.sort(key = lambda x : x[1], reverse=True)
    generation_number += 1
    generations.append( current_generation )
    
    # Generating the new population from the fittest individuals
    selected_genomes = np.full( (100 , 24) , 'sleep')
    for l in range(20):
        for m in range(5):
            for n in range(24):
                a = current_generation[l][2][n]
                a = mutation_gene( a , p )
                selected_genomes[int(100/20)*l + m] [n] = a

Wall time: 2min 34s


In [6]:
%%time
for run_generations in range (5):
    # Creating the Agent-Based models for the current generation and running it for 7 days.
    for current_exp_id in range(100*generation_number + 0, 100*generation_number + 100):
        i = current_exp_id
        abm_models.append( SleepAnimals(i, genome=selected_genomes[i-100*generation_number], 
                                        width=40, height=40) )
        for j in range(60*24*7):
            abm_models[i].step()
    
    # Capturing the data from the end of the simulation
    current_generation = []
    for k in range(100*generation_number + 0, 100*generation_number + 100):
        a = [ abm_models[k].model_id , abm_models[k].schedule.agents[0].fitness , 
             abm_models[k].schedule.agents[0].circadian_rythm ]
        current_generation.append( a )
    
    generation_results = dataframe_generation(generation_number , current_generation)
    results = results.append(generation_results)
    current_generation.sort(key = lambda x : x[1], reverse=True)
    generation_number += 1
    generations.append( current_generation )
    
    # Generating the new population from the fittest individuals
    selected_genomes = np.full( (100 , 24) , 'sleep')
    for l in range(20):
        for m in range(5):
            for n in range(24):
                a = current_generation[l][2][n]
                a = mutation_gene( a , p )
                selected_genomes[int(100/20)*l + m] [n] = a

Wall time: 1min 50s


In [7]:
%%time
for run_generations in range (5):
    # Creating the Agent-Based models for the current generation and running it for 7 days.
    for current_exp_id in range(100*generation_number + 0, 100*generation_number + 100):
        i = current_exp_id
        abm_models.append( SleepAnimals(i, genome=selected_genomes[i-100*generation_number], 
                                        width=40, height=40) )
        for j in range(60*24*7):
            abm_models[i].step()
    
    # Capturing the data from the end of the simulation
    current_generation = []
    for k in range(100*generation_number + 0, 100*generation_number + 100):
        a = [ abm_models[k].model_id , abm_models[k].schedule.agents[0].fitness , 
             abm_models[k].schedule.agents[0].circadian_rythm ]
        current_generation.append( a )
    
    generation_results = dataframe_generation(generation_number , current_generation)
    results = results.append(generation_results)
    current_generation.sort(key = lambda x : x[1], reverse=True)
    generation_number += 1
    generations.append( current_generation )
    
    # Generating the new population from the fittest individuals
    selected_genomes = np.full( (100 , 24) , 'sleep')
    for l in range(20):
        for m in range(5):
            for n in range(24):
                a = current_generation[l][2][n]
                a = mutation_gene( a , p )
                selected_genomes[int(100/20)*l + m] [n] = a

Wall time: 2min 54s


In [ ]:
generation_number , current_exp_id

### Some results

#### 1st generation data

This are the models with the highest fitness in the first generation:

In [ ]:
gen1_mostfit = [ generations[0][0][0] , generations[0][1][0], generations[0][2][0], 
               generations[0][3][0] , generations[0][4][0]]
gen1_mostfit

This shows the genome of the 10 most fit individuals of the first generation, color coded (Green = Eat genome, Lila = Sleep genome, Light blue = flexible genome)

In [ ]:
genome_display_1gen = displayRGB_generation(1, 10, generations)
_ = plt.imshow(np.swapaxes(genome_display_1gen,0,1))

Plotting the "Fitness vs Timestep" of the five most fit agents.

In [ ]:
results_1stgen_1st = abm_models[gen1_mostfit[0]].datacollector.get_agent_vars_dataframe()
results_1stgen_2st = abm_models[gen1_mostfit[1]].datacollector.get_agent_vars_dataframe()
results_1stgen_3rd = abm_models[gen1_mostfit[2]].datacollector.get_agent_vars_dataframe()
results_1stgen_4th = abm_models[gen1_mostfit[3]].datacollector.get_agent_vars_dataframe()
results_1stgen_5th = abm_models[gen1_mostfit[4]].datacollector.get_agent_vars_dataframe()
_ = results_1stgen_1st.Fitness.plot()
_ = results_1stgen_2st.Fitness.plot()
_ = results_1stgen_3rd.Fitness.plot()
_ = results_1stgen_4th.Fitness.plot()
_ = results_1stgen_5th.Fitness.plot()

Dataframe of the 1st generation showing: 20 most fit agents, fitness in the last step and genome.

In [ ]:
results.loc['1st'].nlargest(20,'fitness')

#### 10th generation data

This are the models with the highest fitness in the second generation:

In [ ]:
gen10_mostfit = [ generations[9][0][0] , generations[9][1][0], generations[9][2][0], 
               generations[9][3][0] , generations[9][4][0]]
gen10_mostfit

This shows the genome of the 10 most fit individuals of the tenth generation, color coded (Green = Eat genome, Lila = Sleep genome, Light blue = flexible genome)

In [ ]:
genome_display_10gen = displayRGB_generation(10, 10, generations)
_ = plt.imshow(np.swapaxes(genome_display_10gen,0,1))

Plotting the "Fitness vs Timestep" of the five most fit agents.

In [ ]:
results_10gen_1st = abm_models[gen10_mostfit[0]].datacollector.get_agent_vars_dataframe()
results_10gen_2st = abm_models[gen10_mostfit[1]].datacollector.get_agent_vars_dataframe()
results_10gen_3rd = abm_models[gen10_mostfit[2]].datacollector.get_agent_vars_dataframe()
results_10gen_4th = abm_models[gen10_mostfit[3]].datacollector.get_agent_vars_dataframe()
results_10gen_5th = abm_models[gen10_mostfit[4]].datacollector.get_agent_vars_dataframe()
_ = results_10gen_1st.Fitness.plot()
_ = results_10gen_2st.Fitness.plot()
_ = results_10gen_3rd.Fitness.plot()
_ = results_10gen_4th.Fitness.plot()
_ = results_10gen_5th.Fitness.plot()

Dataframe of the 10th generation showing: 20 most fit agents, fitness in the last step and genome.

In [ ]:
results.loc['10th'].nlargest(20,'fitness')